In [ ]:
import os
import cv2 as cv
import numpy as np

PATH_TO_DATA = os.path.join(".\..\..\Coursework\Datastore\Supplied")
FILE_EXTENTION = ".avi" 

# Creates a list of labels, sourced from each child directory names in the specified path 
def getLabels():
    directories = []
    try:
        for item in os.listdir(PATH_TO_DATA):
            item_path = os.path.join(PATH_TO_DATA, item)
            if os.path.isdir(item_path):
                directories.append(item)                
    except Exception as e:
        print("ERROR, No data found\n\n",e)
    else:
        print("Labels:", directories)
    return directories

#Creates a list of integers which corrolate with the labels list, 
def enumerateLabels(labels):
    LabelIntegers = [i for i in range(len(labels))] # list aprehention

    if LabelIntegers== []:
        print("CANNOT ENumerate, no labels found\n\n")    
    else:
        print("Labels enumerated:",LabelIntegers,"\n")
        
    return LabelIntegers        
    
#creates a 2D array of paths, the first index corrolates with each label integer    
def getFilePaths(labels, PATH_TO_DATA):
    all_data=[]  
    for label in labels:
        label_data=[]
        label_path = os.path.join(PATH_TO_DATA,label)

    
        for item in os.listdir(label_path):
            item_path = os.path.join(label_path, item)    
            if item_path.endswith(FILE_EXTENTION):
                label_data.append(os.path.join(label_path,item))
        print("Found",len(label_data),"Files ending in",FILE_EXTENTION,"for Label,",label)
        all_data.append(label_data)            
    return all_data

def getLabelNameFromInteger(label_int,labels):
    return labels[label_int]

def getLabelIntFromName(label_name,labels):
    return labels.index(label_name)

def getFramesGreyscale(video_path):
    # Open the video file
    
    cap = cv.VideoCapture(video_path)
    frames=[]

    # Check if the video file was successfully opened
    if not cap.isOpened():
        print("Error opening video file",video_path)
        return   

    while True:
        success, frame = cap.read()
        if not success:
            break
        frame = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
        frames.append(frame)     

    cap.release()

    print("Frames extracted from ",video_path)
    
    return frames

def getMHIFromFilePathArray(filePathArray):   
    MHI_array=[]
    for row in filePathArray:
        Label_MHI=[]
        for path in row:
            frames=getFramesGreyscale(path)
            MHI=getMHIFromVideo(frames)
            Label_MHI.append(MHI)
        MHI_array.append(Label_MHI)
            # Perform further processing on frames as needed
            # Example: Save frames to disk, analyze frames, etc.
            
    return MHI_array
        
def getMHIFromVideo(frames):
    # Parameters for motion history calculation
    mhi_duration = 1.0
    max_time_delta = 0.5
    min_time_delta = 0.05
    threshold_value = 25

    # Initialize motion history image (MHI)
    height, width = frames[0].shape
    MHI = np.zeros((height, width), dtype=np.float32)

    for i in range(1, len(frames)):
        # Calculate time interval between frames
        time_delta = 1.0 / len(frames)

        # Calculate motion image
        frame_diff = cv.absdiff(frames[i], frames[i - 1])
        _, motion_mask = cv.threshold(frame_diff, threshold_value, 1, cv.THRESH_BINARY)
        cv.motempl.updateMotionHistory(motion_mask, MHI, time_delta, mhi_duration)

        # Display MHI image until 'q' is pressed

    while True:
    
        # Display the result
        cv.imshow("MHI", MHI)
        if cv.waitKey(1) & 0xFF == ord('q'):  # Quit when 'q' is pressed
            cv.destroyAllWindows()
            break

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
        return MHI

    cv.destroyAllWindows()


def showChannelfromVideoFrames(video, channel_index):
    print("showing Channel",channel_index,"from video frames")
    for frame in video:
        # Split the frame into color channels
        channels = cv.split(frame)

        # Extract the specified channel based on the index
        channel = channels[channel_index]
        Windowsname="Channel"+str(channel_index)
        # Display the channel
        cv.imshow(Windowsname, channel)

        # Check for 'q' key press to quit
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

    cv.destroyAllWindows()

def showVideo(videopath):
    print("showing Video",videopath)
    video = cv.VideoCapture(videopath)
    showShape=True
    while True:
        success, frame = video.read()
        if success:
            if showShape:
                print(frame.shape)
                showShape=False
                # Extract the specified channel based on the index
            Windowsname="Video"+str(videopath)
            # Display the channel
            cv.imshow(Windowsname,frame)

            # Check for 'q' key press to quit
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            video.release()
            cv.destroyAllWindows()
            break


def showChannelFromPath(video_path, channel_index):
    print("showing Channel",channel_index,"from path",video_path)
    
    # Open the video file
    cap = cv.VideoCapture(video_path)

    # Check if the video file was successfully opened
    if not cap.isOpened():
        print("Error opening video file", video_path)
        return
    showShape=True
    while True:
        success, frame = cap.read()
        if success: 
            # Split the frame into color channels
            channels = cv.split(frame)

            # Extract the specified channel based on the index
            channel = channels[channel_index]

            if showShape:
                    print(channel.shape)
                    showShape=False
            
            # Display the channel
            cv.imshow("Channel", channel)

            # Check for 'q' key press to quit
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("releasing video")
            cap.release()
            cv.destroyAllWindows()
            break
            
    
def main():
    labels=(getLabels())
    Label_Integers=enumerateLabels(labels) #works
    filePathArray = getFilePaths(labels, PATH_TO_DATA)    
    
    
    print(filePathArray[0][0])
    
    GreyVideoFrames = getFramesGreyscale(filePathArray[0][0])
    
    showChannelfromVideoFrames(GreyVideoFrames, 0)
    showVideo(filePathArray[0][0])
    
    showChannelFromPath(filePathArray[0][0],0)
    showChannelFromPath(filePathArray[0][0],1)
    showChannelFromPath(filePathArray[0][0],2)
   
    cv.destroyAllWindows()
    
    print("DONE - exiting program")
if __name__=="__main__":
    main()

